# Data Wrangling and Preparation for the Weather Prediction Project

For this weather prediction project, we shall rely on historic hourly weather data available from the US Govenment. It is available publicly and free of change: https://www.ncdc.noaa.gov/cdo-web/datatools/lcd

The documentation, as provided by the US Government: https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/LCD_documentation.pdf

We shall be using 10 years of data (2011-2020) for both the Target Location (Chicago, IL) and for a number of nearby locations in the US Midwest in the states of Illinois, Missoury, Wisconsin, Minnesota, Michigan, Indiana and Ohio.

In [6]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv("../raw-data/noaa_2011-2020_chicago.csv", low_memory=False, parse_dates=['DATE'])
print(raw_df.columns.values)

['STATION' 'DATE' 'REPORT_TYPE' 'SOURCE' 'AWND' 'BackupDirection'
 'BackupDistance' 'BackupDistanceUnit' 'BackupElements' 'BackupElevation'
 'BackupElevationUnit' 'BackupEquipment' 'BackupLatitude'
 'BackupLongitude' 'BackupName' 'CDSD' 'CLDD' 'DSNW'
 'DailyAverageDewPointTemperature' 'DailyAverageDryBulbTemperature'
 'DailyAverageRelativeHumidity' 'DailyAverageSeaLevelPressure'
 'DailyAverageStationPressure' 'DailyAverageWetBulbTemperature'
 'DailyAverageWindSpeed' 'DailyCoolingDegreeDays'
 'DailyDepartureFromNormalAverageTemperature' 'DailyHeatingDegreeDays'
 'DailyMaximumDryBulbTemperature' 'DailyMinimumDryBulbTemperature'
 'DailyPeakWindDirection' 'DailyPeakWindSpeed' 'DailyPrecipitation'
 'DailySnowDepth' 'DailySnowfall' 'DailySustainedWindDirection'
 'DailySustainedWindSpeed' 'DailyWeather' 'HDSD' 'HTDD' 'HeavyFog'
 'HourlyAltimeterSetting' 'HourlyDewPointTemperature'
 'HourlyDryBulbTemperature' 'HourlyPrecipitation'
 'HourlyPresentWeatherType' 'HourlyPressureChange'
 'HourlyPres

We can see that most of these columns are not what we need (which is Hourly data). We also know that `REPORT_TYPE` of `SOD` indicates Daily Data in which we are not interested. Throw away the extra columns and the reports that are not hourly as follows:

In [7]:
raw_df = raw_df[raw_df['REPORT_TYPE'].str.strip() != 'SOD' ]
columns_to_drop = list(filter(lambda c: c != 'DATE' and not c.startswith('Hourly'), raw_df.columns.tolist()))
raw_df = raw_df.drop(columns = columns_to_drop)

print(raw_df.columns.values)

['DATE' 'HourlyAltimeterSetting' 'HourlyDewPointTemperature'
 'HourlyDryBulbTemperature' 'HourlyPrecipitation'
 'HourlyPresentWeatherType' 'HourlyPressureChange'
 'HourlyPressureTendency' 'HourlyRelativeHumidity'
 'HourlySeaLevelPressure' 'HourlySkyConditions' 'HourlyStationPressure'
 'HourlyVisibility' 'HourlyWetBulbTemperature' 'HourlyWindDirection'
 'HourlyWindGustSpeed' 'HourlyWindSpeed']


There, much more manageable!

Before we go ahead further, though, let's keep only the columns that we may need for future modeling. Based on basic knowledge of weather we can see that some of the data is mutually redundant:

- 3 kinds of pressure (keep `AltimeterSetting` only)
- 2 kinds of temperature (keep `DryBulb` only)
- 2 kinds of pressure change (keep `PressureChange` only)



In [19]:
raw_df = raw_df.drop(columns = ['HourlyPressureTendency', 'HourlySeaLevelPressure', 
                                'HourlyStationPressure', 'HourlyWetBulbTemperature'])

Let's take a look at the remaing data:

In [20]:
raw_df.describe(include = 'all', datetime_is_numeric=True)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyRelativeHumidity,HourlySkyConditions,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
count,125865,111200,125696,125707,99058,30244,42934,125693.000000,115054,125714,125523,20952.000000,125662.000000
unique,NaN,209,114,136,179,386,77,NaN,26640,43,38,NaN,NaN
top,NaN,29.98,30,34,0.00,BR:1 ||,+0.01,NaN,CLR:00,10.00,000,NaN,NaN
freq,NaN,2347,2832,2871,73356,5454,5477,NaN,6877,79622,7072,NaN,NaN
mean,2016-01-10 12:40:13.828625664,NaN,NaN,NaN,NaN,NaN,NaN,70.866063,NaN,NaN,NaN,26.545294,9.917119
min,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,14.000000,0.000000
25%,2013-07-24 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,59.000000,NaN,NaN,NaN,22.000000,6.000000
50%,2016-01-17 23:51:00,NaN,NaN,NaN,NaN,NaN,NaN,73.000000,NaN,NaN,NaN,25.000000,9.000000
75%,2018-07-06 14:51:00,NaN,NaN,NaN,NaN,NaN,NaN,85.000000,NaN,NaN,NaN,30.000000,13.000000
max,2020-12-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,NaN,70.000000,55.000000


We can see that the stats are incomplete due to the abundance of missing values. Let's take a look at some rows:

In [18]:
pd.set_option('display.max_rows', 100)
raw_df.head(100)

,DATE,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyPressureChange,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,2011-01-01 00:00:00,NaN,35,42,NaN,NaN,-0.04,3.0,76.0,29.73,NaN,29.03,9.94,39.0,260,26.0,15.0
1,2011-01-01 00:49:00,29.72,30,39,NaN,NaN,NaN,NaN,70.0,NaN,BKN:07 26 OVC:08 37,29.00,10.00,35.0,250,NaN,11.0
2,2011-01-01 00:51:00,29.72,30,40,0.00,NaN,NaN,NaN,68.0,29.73,BKN:07 26 OVC:08 37,29.00,10.00,36.0,240,NaN,13.0
3,2011-01-01 01:18:00,29.73,28,37,NaN,NaN,NaN,NaN,70.0,NaN,OVC:08 30,29.01,10.00,33.0,260,NaN,17.0
4,2011-01-01 01:51:00,29.74,28,37,0.00,NaN,NaN,NaN,70.0,29.75,BKN:07 32,29.02,10.00,33.0,260,22.0,17.0
5,2011-01-01 02:51:00,29.75,27,36,0.00,NaN,-0.02,3.0,70.0,29.75,OVC:08 43,29.03,10.00,33.0,240,24.0,17.0
6,2011-01-01 03:51:00,29.75,20,32,0.00,NaN,NaN,NaN,61.0,29.76,BKN:07 48,29.03,10.00,28.0,240,25.0,15.0
7,2011-01-01 04:51:00,29.76,19,31,0.00,NaN,NaN,NaN,61.0,29.77,SCT:04 48,29.04,10.00,27.0,240,25.0,16.0
8,2011-01-01 05:51:00,29.77,17,28,0.00,NaN,-0.02,3.0,63.0,29.78,FEW:02 48,29.05,10.00,24.0,240,28.0,18.0
9,2011-01-01 06:00:00,NaN,17,28,NaN,NaN,-0.02,3.0,63.0,29.78,NaN,29.07,9.94,24.0,240,28.0,18.0


Yes, there are quite a few NANs as well as funny characters in Numeric columns. Much imputation is needed to clean this up. 

We shall do that step by step:

In [26]:
# Precipitation: replace 'T' ("trace amount") with a small value
raw_df.loc[raw_df['HourlyPrecipitation'].str.strip() == 'T', 'HourlyPrecipitation'] = 0.005


In [27]:
# Put NANs in for variables that are meant to be Numeric but aren't
def removeJunkSuffixesAndPrefixes(df, variables):
    for variable in variables:
        if df[variable].dtypes != np.float64:
            df.loc[df[variable].str.contains('s', na=False), variable] = ''
            df.loc[df[variable].str.contains('V', na=False), variable] = ''
            df.loc[df[variable].str.contains('\\*', na=False), variable] = ''

    return df

numeric_variables = ['HourlyDryBulbTemperature', 'HourlyRelativeHumidity', 'HourlyWindSpeed', 'HourlyWindDirection', 'HourlyDewPointTemperature', 
    'HourlyWindGustSpeed', 'HourlyAltimeterSetting', 'HourlyPressureChange', 'HourlyPrecipitation', 'HourlyPressureChange', 'HourlyVisibility']
raw_df = removeJunkSuffixesAndPrefixes(raw_df, numeric_variables)